<div class = "alert alert-block alert-info">
    <h1><font color = "red">DISCLAIMER</font></h1>
    <p>The following notebook it's highly based on the works <a href = "https://www.kaggle.com/optimo/tabnetregressor-2-0-train-infer">TabNetRegressor 2.0 [TRAIN + INFER]</a>, <a href = "https://www.kaggle.com/liuhdme/moa-competition/data">MOA competition</a> and <a href = "https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn/data?select=train_targets_scored.csv">
MoA | Pytorch | 0.01859 | RankGauss | PCA | NN</a>, please check it out. I have to add that i don't make this notebook for "upvotes" but feedback.</p>
</div>

# <font color = "seagreen">Preambule</font>

I made this notebook to share some experiments (see the sections "Experiments") which could help to someone who don't want to wast their daily "submissions", but more importantly, to get feedback about what i could change to achive a better CV. Moreover, the easiness of TabNet to overfit the data it's disturbing. In the section "Conclusion" i share my opinion about the fine-tuning process of TabNet.

## <font color = "green">Installing Libraries</font>

In [ ]:
# TabNet
# !pip install pytorch-tabnet
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet
# Iterative Stratification
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

## <font color = "green">Loading Libraries</font>

In [ ]:
### General ###
import os
import sys
import copy
import tqdm
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
sys.path.append("../input/rank-gauss")
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

### Data Wrangling ###
import numpy as np
import pandas as pd
from scipy import stats
from gauss_rank_scaler import GaussRankScaler

### Data Visualization ###
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

### Machine Learning ###
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

### Deep Learning ###
import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Tabnet 
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN

## <font color = "green">Reproducibility</font>

In [ ]:
seed = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(seed)

## <font color = "green">Configuration</font>

In [ ]:
# Parameters
data_path = "../input/lish-moa/"
no_ctl = True
scale = "rankgauss"
variance_threshould = 0.7
decompo = "PCA"
ncompo_genes = 80
ncompo_cells = 10
encoding = "dummy"

In [ ]:
train = pd.read_csv(data_path + "train_features.csv")
targets = pd.read_csv(data_path + "train_targets_scored.csv")
test = pd.read_csv(data_path + "test_features.csv")
train_drug = pd.read_csv(data_path + "train_drug.csv")
submission = pd.read_csv(data_path + "sample_submission.csv")
print(train.shape,test.shape)
if no_ctl:
    # cp_type == ctl_vehicle
    print(b_, "not_ctl")
    train = train[train["cp_type"] != "ctl_vehicle"]
    test = test[test["cp_type"] != "ctl_vehicle"]
    targets = targets.iloc[train.index]
    train.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    targets.reset_index(drop = True, inplace = True)
print('Non-control:', train.shape,test.shape)

cols_numeric = [feat for feat in list(train.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]
mask = (train[cols_numeric].var() >= variance_threshould).values
tmp = train[cols_numeric].loc[:, mask]
train = pd.concat([train[["sig_id", "cp_type", "cp_time", "cp_dose"]], tmp], axis = 1)
cols_numeric = [feat for feat in list(train.columns) if feat not in ["sig_id", "cp_type", "cp_time", "cp_dose"]]
test = pd.concat([test[["sig_id", "cp_type", "cp_time", "cp_dose"]], test.loc[:,cols_numeric]], axis = 1)
print('Feature elimination by variance :', train.shape,test.shape)


GENES = [col for col in train.columns if col.startswith("g-")]
CELLS = [col for col in train.columns if col.startswith("c-")]




if scale == "rankgauss":
    print(b_, "Rank Gauss")
    for col in (GENES + CELLS):
        transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")   # from optimal commit 9
        vec_len = len(train[col].values)
        vec_len_test = len(test[col].values)
        raw_vec = train[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)
        
        train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
        test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
else:
    pass
print("Rank Gauss:", train.shape,test.shape)

if decompo == "PCA":
    print(b_, "PCA")
    GENES = [col for col in train.columns if col.startswith("g-")]
    CELLS = [col for col in train.columns if col.startswith("c-")]
    
    pca_genes = PCA(n_components = ncompo_genes, random_state = seed)
    pca_genes_train = pca_genes.fit_transform(train[GENES])
    
    
    
    pca_cells = PCA(n_components = ncompo_cells, random_state = seed)
    pca_cells_train = pca_cells.fit_transform(train[CELLS])
    
    pca_genes_train = pd.DataFrame(pca_genes_train, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
    pca_cells_train = pd.DataFrame(pca_cells_train, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
    train = pd.concat([train, pca_genes_train, pca_cells_train], axis = 1)
    
    pca_genes_test = pca_genes.transform(test[GENES])
    pca_cells_test = pca_cells.transform(test[CELLS])
    
    pca_genes_test = pd.DataFrame(pca_genes_test, columns = [f"pca_g-{i}" for i in range(ncompo_genes)])
    pca_cells_test = pd.DataFrame(pca_cells_test, columns = [f"pca_c-{i}" for i in range(ncompo_cells)])
    test = pd.concat([test, pca_genes_test, pca_cells_test], axis = 1)
else:
    pass
print("PCA:", train.shape,test.shape)


if encoding == "dummy":
    print(b_, "One-Hot")
    train = pd.get_dummies(train, columns = ["cp_time", "cp_dose"])
    test = pd.get_dummies(test, columns = ["cp_time", "cp_dose"])
else:
    pass
print("One Hot:", train.shape,test.shape)


for stats in tqdm.tqdm(["sum", "mean", "std", "kurt", "skew"]):
    train["g_" + stats] = getattr(train[GENES], stats)(axis = 1)
    train["c_" + stats] = getattr(train[CELLS], stats)(axis = 1)    
    train["gc_" + stats] = getattr(train[GENES + CELLS], stats)(axis = 1)
    
    test["g_" + stats] = getattr(test[GENES], stats)(axis = 1)
    test["c_" + stats] = getattr(test[CELLS], stats)(axis = 1)    
    test["gc_" + stats] = getattr(test[GENES + CELLS], stats)(axis = 1)
print('Extra features:', train.shape,test.shape)

gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577',
             'g-153','g-389','g-60','g-370','g-248','g-167',
             'g-203','g-177','g-301','g-332','g-517','g-6',
             'g-744','g-224','g-162','g-3','g-736','g-486',
             'g-283','g-22','g-359','g-361','g-440','g-335',
             'g-106','g-307','g-745','g-146','g-416','g-298',
             'g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

for df in [train, test]:
    df['c52_c42'] = df['c-52'] * df['c-42']
    df['c13_c73'] = df['c-13'] * df['c-73']
    df['c26_c13'] = df['c-23'] * df['c-13']
    df['c33_c6'] = df['c-33'] * df['c-6']
    df['c11_c55'] = df['c-11'] * df['c-55']
    df['c38_c63'] = df['c-38'] * df['c-63']
    df['c38_c94'] = df['c-38'] * df['c-94']
    df['c13_c94'] = df['c-13'] * df['c-94']
    df['c4_c52'] = df['c-4'] * df['c-52']
    df['c4_c42'] = df['c-4'] * df['c-42']
    df['c13_c38'] = df['c-13'] * df['c-38']
    df['c55_c2'] = df['c-55'] * df['c-2']
    df['c55_c4'] = df['c-55'] * df['c-4']
    df['c4_c13'] = df['c-4'] * df['c-13']
    df['c82_c42'] = df['c-82'] * df['c-42']
    df['c66_c42'] = df['c-66'] * df['c-42']
    df['c6_c38'] = df['c-6'] * df['c-38']
    df['c2_c13'] = df['c-2'] * df['c-13']
    df['c62_c42'] = df['c-62'] * df['c-42']
    df['c90_c55'] = df['c-90'] * df['c-55']
    
    for feature in gsquarecols:
        if feature in GENES:
            df[f'{feature}_squared'] = df[feature] ** 2  
    for feature in CELLS:
        df[f'{feature}_squared'] = df[feature] ** 2   

train = train.merge(targets, on='sig_id')
train = train.merge(train_drug, on='sig_id')

target_cols = [x for x in targets.columns if x != 'sig_id']

train_df = train
train_df.reset_index(drop = True, inplace = True)
test.reset_index(drop = True, inplace = True)
features_to_drop = ["sig_id", "cp_type"]

test.drop(features_to_drop, axis = 1, inplace = True)
X_test = test.values

In [ ]:
pd.set_option('display.max_columns', None)
print('Train_df shape:', train_df.shape)
print('Test shape:', test.shape)
display(train_df.head())
display(test.head())


In [ ]:
feature_cols = [c for c in train_df.columns if c not in targets.columns]
feature_cols = [c for c in feature_cols if c not in [ 'sig_id', 'drug_id', 'cp_type']]
print(train_df.shape)
display(train_df[feature_cols].head())

In [ ]:
from sklearn.model_selection import KFold

def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in SEEDS:
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = [x for x in range(42,49)]
NFOLDS = 10
DRUG_THRESH = 18

train_df = make_cv_folds(train_df, SEEDS, NFOLDS, DRUG_THRESH)
train_df.head()

In [ ]:
# features_to_drop = ["sig_id", "cp_type"]
# train.drop(features_to_drop, axis = 1, inplace = True)
# test.drop(features_to_drop, axis = 1, inplace = True)
# try:
#     targets.drop("sig_id", axis = 1, inplace = True)
# except:
#     pass

# print(train.shape,test.shape)



In [ ]:
display(train_df.head())
print(train_df.shape)
print(feature_cols)

# <font color = "seagreen">Experiments</font>

I just want to point that the [original work](https://www.kaggle.com/optimo/tabnetregressor-2-0-train-infer) achive a CV of 0.015532370835690834 and a LB score of 0.01864. Some of the experiments that i made with their changes:


- CV: 0.01543560538566987, LB: 0.01858, best LB that i could achive, changes
    - `n_a` = 32 instead of 24
    - `n_d` = 32 instead of 24
- CV: 0.015282077428722094, LB: 0.01862, best CV that i could achive, changes (Version 5):
    - `n_a` = 32 instead of 24
    - `n_d` = 32 instead of 24
    - `virtual_batch_size` = 32, instead of 128
    - `seed` = 42 instead of 0
- CV: 0.015330138325308062, LB: 01864, the same LB that the original but better CV, changes:
    - `n_a` = 32 instead of 24
    - `n_d` = 32 instead of 24
    - `virtual_batch_size` = 64, instead of 128
    - `batch_size` = 512, instead of 1024
- CV: 0.015361751699863063, LB: 0.01863, better LB and CV than the original, changes:
    - `n_a` = 32 instead of 24
    - `n_d` = 32 instead of 24
    - `virtual_batch_size` = 64, instead of 128
- CV: 0.015529925324634975, LB: 0.01865, changes:
    - `n_a` = 48 instead of 24
    - `n_d` = 48 instead of 24
- CV: 0.015528553520924939, LB: 0.01868, changes:
    - `n_a` = 12 instead of 24
    - `n_d` = 12 instead of 24
- CV: 0.015870202970324317, LB: 0.01876, worst CV and LB score, changes:
    - `n_a` = 12 instead of 24
    - `n_d` = 12 instead of 24
    - `batch_size` = 2048, instead of 1024
    
    
As you can see if `batch_size` < 1024 and > 1024 give worst results. Something similar happens with `n_a` and `n_d`, if their values are lower or higher than 32 the results are worst.


## <font color = "green">Versions</font>

- **Version 5**: I added the `seed` parameter to the TabNet model.
- **Version 6**: I changed the `virtual_batch_size` to 24
    - CV: 0.01532900616425282, LB: 0.01862, changes:
        - `n_a` = 32 instead of 24
        - `n_d` = 32 instead of 24
        - `virtual_batch_size` = 24, instead of 128
        - `seed` = 42 instead of 0
- **Version 7**: PCA, Rank Gauss

# <font color = "seagreen">Modeling</font>

## <font color = "green">Model Parameters</font>

In [ ]:
MAX_EPOCH = 200
# n_d and n_a are different from the original work, 32 instead of 24
# This is the first change in the code from the original
tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 10, min_lr = 1e-5, factor = 0.1),
    scheduler_fn = ReduceLROnPlateau,
    seed = seed,
    verbose = 10
)

## <font color = "green">Custom Metric</font>

In [ ]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

# <font color = "seagreen">Training</font>

In [ ]:
targets.head(2)

In [ ]:
scores_auc_all = []
test_cv_preds = []

oof_preds = []
oof_targets = []
scores = []
scores_auc = []
oof_all = np.zeros((train_df.shape[0], targets[target_cols].shape[1]))
# SEED = [42,43,44] # 
# SEED = [42]
for s in SEEDS:
    oof = np.zeros((train_df.shape[0], targets[target_cols].shape[1]))
    tabnet_params['seed'] = s
    for fold_id in range(NFOLDS):
        print(b_,"FOLDS: ", r_, fold_id + 1, y_, 'seed:', tabnet_params['seed'])
        print(g_, '*' * 60, c_)
        
        kfold_col = f'kfold_{s}'
        trn_idx = train_df[train_df[kfold_col] != fold_id].index
        val_idx = train_df[train_df[kfold_col] == fold_id].index
    
#         train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
#         valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
        
        
        X_train, y_train = train_df[feature_cols].values[trn_idx, :], targets[target_cols].values[trn_idx, :]
        X_val, y_val = train_df[feature_cols].values[val_idx, :], targets[target_cols].values[val_idx, :]
        ### Model ###
        model = TabNetRegressor(**tabnet_params)
        
        ### Fit ###
        # Another change to the original code
        # virtual_batch_size of 32 instead of 128
        model.fit(
            X_train = X_train,
            y_train = y_train,
            eval_set = [(X_val, y_val)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs = MAX_EPOCH,
            patience = 30,
            batch_size = 1024, 
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
            # To use binary cross entropy because this is not a regression problem
            loss_fn = F.binary_cross_entropy_with_logits
        )
        print(y_, '-' * 60)
        
        saving_path_name = 'TabNet_seed_'+str(tabnet_params['seed'])+'_fold_'+str(fold_id+1)
        saved_filepath = model.save_model(saving_path_name)
        print(saved_filepath)

        loaded_model =  TabNetRegressor()
#         loaded_model.load_model(saved_filepath)
        loaded_model.load_model(saving_path_name+'.zip')

        ### Predict on validation ###
#         preds_val = model.predict(X_val)
        preds_val = loaded_model.predict(X_val)
        # Apply sigmoid to the predictions
        preds = 1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
#         loaded_model.history["val_logits_ll"] = model.history["val_logits_ll"]
        
        # Added oof part
        oof[val_idx] = preds
    
        ### Save OOF for CV ###
        oof_preds.append(preds_val)
        oof_targets.append(y_val)
        scores.append(score)
    
        ### Predict on test ###
        preds_test = loaded_model.predict(X_test)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))
    oof_all += oof
    
    
oof_preds_all = np.concatenate(oof_preds)
oof_targets_all = np.concatenate(oof_targets)
test_preds_all = np.stack(test_cv_preds)
oof_all = oof_all/len(SEEDS)

In [ ]:
oof.shape, oof_all.shape

In [ ]:
oof_all[:5]

In [ ]:
print(test_preds_all.shape)
print(test_preds_all[0][:5])

In [ ]:
aucs = []
for task_id in range(oof_preds_all.shape[1]):
    aucs.append(roc_auc_score(y_true = oof_targets_all[:, task_id],
                              y_score = oof_preds_all[:, task_id]
                             ))
print(f"{b_}Overall AUC: {r_}{np.mean(aucs)}")
print(f"{b_}Average CV: {r_}{np.mean(scores)}")

In [ ]:
print(oof_preds_all.shape)
print(oof_targets_all.shape)
print(oof_preds_all.shape)
print(tabnet_params['seed'])

**The worst CV value that i achive**

# <font color = "seagreen">Conclusion (NOT AVAILABLE UNTIL I SEE THE LB Score)</font> 

# <font color = "seagreen">Submission</font>

In [ ]:
print(test_preds_all.shape)

In [ ]:
all_feat = [col for col in submission.columns if col not in ["sig_id"]]
# To obtain the same lenght of test_preds_all and submission
test = pd.read_csv(data_path + "test_features.csv")
sig_id = test[test["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)
tmp = pd.DataFrame(test_preds_all.mean(axis = 0), columns = all_feat)
tmp["sig_id"] = sig_id

submission = pd.merge(test[["sig_id"]], tmp, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)

#submission[all_feat] = tmp.mean(axis = 0)

# Set control to 0
#submission.loc[test["cp_type"] == 0, submission.columns[1:]] = 0
submission.to_csv("submission.csv", index = None)
submission.head()

In [ ]:
print(f"{b_}submission.shape: {r_}{submission.shape}")

<div class = "alert alert-block alert-info">
    <h3><font color = "red">NOTE: </font></h3>
    <p>If you want to comment please tag me with '@' to answer more quickly.</p>
</div>

In [ ]:
!ls *

In [ ]:
np.save('oof_TabNet_no_ctrl.npy', oof_all)

In [ ]:
oof_all.shape

In [ ]:
train_targets_scored = pd.read_csv(data_path + "train_targets_scored.csv")
target_cols = [col for col in train_targets_scored.columns if col != 'sig_id']

In [ ]:
train = pd.read_csv(data_path + 'train_features.csv')
print(train.shape)
train_no_ctrl = train[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
print(train_no_ctrl.shape)
train_pred = pd.DataFrame(oof_all, columns = target_cols)
print(train_pred.shape)
train_pred = pd.concat([train_no_ctrl[['sig_id']], train_pred], axis=1).reset_index(drop=True)
print(train_pred.shape)
display(train_pred.head())

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train_pred, on='sig_id', how='left').fillna(0)
y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values
np.save('oof_TabNet_all.npy', y_pred) 

In [ ]:
!ls *

In [ ]:
def log_loss_numpy(y_pred, y_true):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
    return loss.mean()

In [ ]:
y_true = train_targets_scored[target_cols].values
log_loss_numpy(y_pred, y_true)